In [1]:
from bs4 import BeautifulSoup as bs
import requests as req
import urllib.request
from urllib.parse import quote
import re
import pandas as pd

In [2]:
# get_news()함수에서 넘어온 article을 본문만 정제하여 return한다
def refine_article(article):
    article = str(article)

    # 첫 줄(<div class="go_trans _article_content">)과 마지막 줄(</div>) 지우기
    if article.find('go_trans _article_content') >= 0:
        #article = article[article.find('<', 1) : article.rfind('</div>')-1]
        article = article[article.find('<', 1) : ]
    
    # <strong> 부분 삭제
    strong_idx = article.find('<strong')
    while strong_idx >= 0:
        new_idx = article.find('/strong>') + 8
        article = article[ : strong_idx] + article[new_idx: ]
        strong_idx = article.find('<strong')

    if article.find('<strong') >= 0:
        article = article[article.find('/strong>')+8 : ]

    # <div> 태그가 첫머리에 시작되면 해당태그의 </div>태그까지 지운다
    if article.find('<div') == 0:
        cur_div_idx = 1
        div_cnt = 1
        while div_cnt > 0:
            idx1 = article.find('<div', cur_div_idx)
            idx2 = article.find('div>', cur_div_idx)
            if idx1 > 0 and idx1 < idx2:  # /div> 이전에 <div가 발견되면
                cur_div_idx = idx1 + 1
                div_cnt += 1
            elif idx2 > 0:
                cur_div_idx = idx2 + 4
                div_cnt -= 1

        article = article[cur_div_idx : ]

    # <table> 부분 삭제
    if article.find('<table') >= 0:
        article = article[article.rfind('/table>')+7 : ]
    
    # <b> 부분 삭제
    if article.find('<b>') >= 0:
        article = article[ : article.find('<b>')] + article[article.rfind('/b>')+3 : ]

    # <span> 부분 삭제 - span은 복수개가 존재할 수 있다
    span_idx = article.find('<span')
    while span_idx >= 0:
        new_idx = article.find('/span>') + 6
        article = article[ : span_idx] + article[new_idx: ]
        span_idx = article.find('<span')

    # 본문내용중에 있는 <br/>, <div>, </div>m \t을 삭제한다.
    article = article.replace('<br/>', '')
    article = article.replace('<div>', '')
    article = article.replace('</div>', '')
    article = article.replace('\t', '')

    return article

In [3]:
def get_news(query):

    news_df = pd.DataFrame(columns=('Title', 'Link', 'Press', 'Datetime', 'Article'))
    idx = 0

    url_query = quote(query)
    #url_query = query
    #url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query=' + url_query 
    # 위의 url와 동일함
    url = 'https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query=' + url_query

    search_url = req.get(url).text
    soup = bs(search_url, 'html.parser')

    links = soup.find_all('div', {'class': 'info_group'})

    for link in links:
        atags = link.select('a.info')
        
        title, news_url, press, datetime, article = '', '', '', '', ''
        for atag in atags:
            # atags가 1개이면 'naver 뉴스'링크가 없으므로 별도의 로직을 구현한다
            if len(atags) == 1:
                news_area = link.parent.parent
               # 언론사
                press = link.find('a', 'info press').text

                # 기사입력시간
                datetime = link.find('span', 'info').text

                # 기사 제목
                title = news_area.find('a', 'news_tit').text

                # 링크
                news_url = news_area.find('a', 'btn_save _keep_trigger').get('data-url')
 
                # 기사내용
                article = news_area.find('a', 'api_txt_lines dsc_txt_wrap').text
                
            else:
                if str(atag).find('info press') >= 0:  # class가 'info press'면 언론사명을 취한다
                    press = atag.text.replace('언론사 선정', '')  # '언론사 선정'이 있으면 삭제한다
                    continue
                else:
                    news_url = atag.get('href')

                    # 뉴스 가져오기
                    if news_url == '#' :  # url이 없다면(즉, '네이버뉴스' 링크가 없다면 continue)
                        continue
                    elif news_url.find('n.news.naver.com') < 0:  # 일반뉴스가 아니라 스포츠뉴스같은 특정 페이지로 넘어가면
                        news_area = link.parent.parent
                        # 기사 제목
                        title = news_area.find('a', 'news_tit').text

                        # 기사입력시간
                        datetime = link.find('span', 'info').text

                        # 기사내용
                        article = news_area.find('a', 'api_txt_lines dsc_txt_wrap').text
                    else:
                        search_news_url = urllib.request.urlopen(news_url)  # 이유는 알 수 없으나 req.get(news_url)은 오류발생
                        soup_news = bs(search_news_url.read(), 'html.parser')
                        
                        # 기사 제목
                        title = soup_news.find('h2', {'id':'title_area'}).text

                        # 기사입력시간
                        datetime = soup_news.find('span', 'media_end_head_info_datestamp_time').text

                        # 기사내용
                        article = refine_article(soup_news.find('div', 'go_trans _article_content'))  # 본문내용만 추려내는 함수로 보낸다

            # 데이터프레임에 저장
            news_df.loc[idx] = [title, news_url, press, datetime, article]
            idx += 1

    return news_df

In [4]:
df_result = get_news('누리호')
df_result

,Title,Link,Press,Datetime,Article
0,"누리호, 첫 위성 손님 태우고 오늘 우주로…""우주 산업화 도약""",https://n.news.naver.com/mnews/article/001/001...,연합뉴스,2023.05.24. 오전 5:01,(서울=연합뉴스) 나확진 기자 = 국내 독자 기술로 개발된 한국형 발사체 누...
1,발사 앞둔 누리호‥이 시각 나로우주센터,https://n.news.naver.com/mnews/article/214/000...,MBC,2023.05.24. 오전 6:04,[뉴스투데이]◀ 앵커 ▶우리 손으로 만든 발사체 누리호가 이제 몇 시간 뒤면 우주로...
2,'첫 손님' 태우고 떠나는 누리호...오늘 오후 발사 예정,https://n.news.naver.com/mnews/article/052/000...,YTN,2023.05.24. 오전 8:05,"오늘 오후, 누리호가 세 번째 우주비행에 나섭니다.처음으로 손님인 실용 위성도 8기..."
3,"""소낙비에도 끄떡없다"" 누리호 내일 발사 준비 끝(종합)",https://n.news.naver.com/mnews/article/003/001...,뉴시스,2023.05.23. 오후 9:42,[서울=뉴시스]윤현성 기자 = 한국형 발사체 누리호가 3차 발사를 하루 앞두고 발사...
4,누리호 오늘(24일) 우주 간다…오후 6시 24분 발사,https://n.news.naver.com/mnews/article/629/000...,더팩트,2023.05.24. 오전 8:54,한국형 우주발사체 '누리호'가 3차 발사에 나선다.24일 과학기술정보통신부(과기정통...
5,"누리호, 오후 6시 24분 발사 확정",https://n.news.naver.com/mnews/article/009/000...,매일경제,2023.05.24. 오후 2:03,누리호 3차 발사가 당초 목표대로 6시24분으로 확정됐다.과학기술정보통신부는 24일...
6,누리호 기립 완료…오늘 오후 6시 24분경 우주로,https://n.news.naver.com/mnews/article/056/001...,KBS,2023.05.24. 오전 6:04,[앵커] 우리 순수 기술로 만든 발사체 '누리호'가 오늘 역사적인 세 번째 우주 비...
7,발사대 도착한 누리호…오전 중 기립 완료한다,https://n.news.naver.com/mnews/article/023/000...,조선일보,2023.05.23. 오전 9:35,첫 실전 발사를 앞두고 있는 한국형발사체 누리호가 발사대 이송을 완료했다.과학기술정...
8,누리호 오늘 '우주로'‥이 시각 나로우주센터,https://n.news.naver.com/mnews/article/214/000...,MBC,2023.05.24. 오후 12:02,[정오뉴스]◀ 앵커 ▶우리 손으로 만든 우주 발사체 누리호의 세번째 발사가 이제 7...
9,"누리호 8시 54분 발사대 도착…""오전 중으로 기립 완료""",https://n.news.naver.com/mnews/article/055/000...,SBS,2023.05.23. 오전 9:22,"<div style=""text-align:center"">한국형 발사체 누리호가 오늘..."
